<a href="https://colab.research.google.com/github/EpicScizor/hello-world/blob/master/TDT4171_Ex5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import graphviz
import sklearn
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import naive_bayes
from sklearn import tree
import tensorflow.keras as keras
import pickle
import time
import traceback

load_keras_model = False
print("Defining functions")

def	sk_readData():
	data = pickle.load(open("sklearn-data.pickle","rb"))
	# Create a vectorizer with some parameters
	vectorizer = HashingVectorizer(stop_words = None, n_features = 2**20, binary = True)
	data["x_train"] = vectorizer.fit_transform(data.get("x_train"))
	data["x_test"] = vectorizer.fit_transform(data.get("x_test"))
	return data

def sk_classifier(obs,res,obs_test,res_test,method="Naive Bayes"):
	if method == "Naive Bayes":
		classifier = naive_bayes.BernoulliNB()
	elif method == "Tree":
		classifier = tree.DecisionTreeClassifier(max_depth = 10)
	else:
		raise IOError("Error: Incorrect method")
	classifier.fit(X = obs, y = res)
	res_pred = classifier.predict(obs_test)
	accuracy = sklearn.metrics.accuracy_score(res_test, res_pred)
	print(method,"accuracy:",accuracy)
	return classifier
	
def sk_main():
	print("Importing sci-kit learn data")
	data = sk_readData()
	print("Training Naive Bayes")
	start_bayes = time.time()
	my_bayes = sk_classifier(data["x_train"],
							data["y_train"],
							data["x_test"],
							data["y_test"],
							method="Naive Bayes")
	end_bayes = time.time()
	print("Naive Bayes time:",end_bayes-start_bayes)
	print("Training Decision Tree")
	my_tree = sk_classifier(data["x_train"],
							data["y_train"],
							data["x_test"],
							data["y_test"],
							method="Tree")
	end_tree = time.time()
	print("Tree time:",end_tree-end_bayes)
	return [my_bayes,my_tree]

def k_readData(maxlen = 200):
	data = pickle.load(open("keras-data.pickle","rb"))
	data["x_train"] = keras.preprocessing.sequence.pad_sequences(data["x_train"],maxlen=maxlen,padding='post',truncating='post')
	data["y_train"] = np.stack((np.asarray(data["y_train"]),1-np.asarray(data["y_train"])),axis=1)
	data["x_test"] = keras.preprocessing.sequence.pad_sequences(data["x_test"],maxlen=maxlen,padding='post',truncating='post')
	data["y_test"] = np.stack((np.asarray(data["y_test"]),1-np.asarray(data["y_test"])),axis=1)
	return data

def inspectData():
	data = pickle.load(open("keras-data.pickle","rb"))
	xlen = np.array(range(0,np.shape(data["x_train"])[0]))
	k = 0
	for i in data["x_train"]:
		c[k] = len(i)
		k = k+1
	plt.hist(c,bins = 50)
	plt.show()
	
def defineModel(input_dim = 1000,embed_dim = 32,input_length = 256,hidden_nodes = 32):	
	model = keras.Sequential()
	model.add(keras.layers.Embedding(input_dim = input_dim,
									output_dim = embed_dim,
									input_length = input_length))
	model.add(keras.layers.LSTM(hidden_nodes,dropout=0.2,recurrent_dropout=0.2))
	model.add(keras.layers.Dense(2,activation='sigmoid'))
	model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
	print(model.summary())
	return model

def evaluateKeras(data,model,batch_size = 32,epochs = 10):
	early_stopping_monitor = keras.callbacks.EarlyStopping(patience=2)
	model.fit(data["x_train"],
			data["y_train"],
			batch_size = batch_size,
			epochs = epochs,
			callbacks = [early_stopping_monitor])
	accuracy = model.evaluate(data["x_test"],data["y_test"])
	print("Accuracy:",accuracy)
	return model
	
def k_main():
	## Hyperparameters
	embed_dim = 32 # dimension of embedded dense vector
	hidden_nodes = 32 # amount of hidden nodes in LSTM
	batch_size = 32
	epochs = 10
	input_length = 256
	
	## Read data from file
	print("Importing keras-data")
	data = k_readData(maxlen = input_length)

	vocab_size = data["vocab_size"]
	
	if load_keras_model:
		print("Loading keras model from file")
		model = load_model('saved_keras_model.h5')
	else:
		print("Defining keras model")
		model = defineModel(input_dim = vocab_size,embed_dim = embed_dim,input_length = input_length)
		print("Training keras model")
		start = time.time()
		model = evaluateKeras(data,model,batch_size,epochs)
		end = time.time()
		print("Recurrent Neural Network time:",end-start)
		model.save('saved_keras_model.h5')
	return model

print("Executing main scripts")
#sk_main()
k_main()
input("Press enter to close")

Defining functions
Executing main scripts
Importing keras-data


FileNotFoundError: ignored